In [1]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth
from PIL import Image



In [2]:
import csv as cs

shipData = []
with open('C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\ships-lat-long.csv') as csv_file:
    csv = cs.reader(csv_file, delimiter=',')
    for row in csv:
        shipData.append(row)

noShipData = []
with open('C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\noships-lat-long.csv') as csv_file:
    csv = cs.reader(csv_file, delimiter=',')
    for row in csv:
        noShipData.append(row)
print()

In [ ]:
# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

image_count = 1
for ship in shipData[1:]:
  geo_geom = {
        "type": "Point",
        "coordinates": [float(ship[2]),float(ship[3])]
  }
  # get images that overlap with our AOI 
  geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_geom
  }
  #2023-06-27T08:33:48.000Z
  time = ship[1]
  start = time[:10] + "T00:00:00.000Z"
  end = time[:10] + "T23:59:59.000Z"
  # get images acquired within a date range
  date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start,
        "lte": end
    }
  }

  # combine our geo, date, cloud filters
  combined_filter = {
    "type": "AndFilter",
    #"config": [geometry_filter]

    "config": [geometry_filter, date_range_filter]
  }

  item_type = "PSScene"

  # API request object
  search_request = {
    "item_types": [item_type], 
    "filter": combined_filter
  }

  # fire off the POST request
  search_result = \
    requests.post(
      'https://api.planet.com/data/v1/quick-search',
      auth=HTTPBasicAuth(API_KEY, ''),
      json=search_request)

  geojson = search_result.json()


  # extract image IDs only
  image_ids = [feature['id'] for feature in geojson['features']]
  if len(image_ids) > 0:
    print(image_ids)
    # For demo purposes, just grab the first image ID
    id0 = image_ids[0]
    id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id0_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    print(result.json().keys())

    print(result.json()['ortho_visual']['status'])
    # Parse out useful links
    links = result.json()[u"ortho_visual"]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the 'ortho_analytic_4b' asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
    )
    activation_status_result = \
    requests.get(
      self_link,
      auth=HTTPBasicAuth(API_KEY, '')
    )
    print(activation_status_result.json)
    print(activation_status_result.json()["status"])
    # Image can be downloaded by making a GET with your Planet API key, from here:
    download_link = activation_status_result.json()["location"]
    print(download_link)

    url = download_link
    myfile = requests.get(url)    
    'c:/users/shash/downloads/PythonImage21.tif'
    lat = round(geo_geom["coordinates"][0], 3)
    long = round(geo_geom["coordinates"][1], 3)
    file_name = "ship_" + str(lat) + "_" + str(long) + "_" + str(start[:10])
    open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\"+file_name+".tif", 'wb').write(myfile.content)
    """tiff_image = Image.open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".tif")
    png = tiff_image.convert('RGB')
    png = png.convert('L')
    png.show()
    png.save("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".png", "PNG")"""
    image_count += 1
    print(image_count)


In [ ]:
# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

image_count = 1
index = 0
for no_ship in noShipData[1:]:
  geo_geom = {
        "type": "Point",
        "coordinates": [float(no_ship[18]),float(no_ship[17])]
  }
  # get images that overlap with our AOI 
  geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_geom
  }
  #2023-06-27T08:33:48.000Z
  time = str(no_ship[1])[:10]
  start = time[:10] + "T00:00:00.000Z"
  end = time[:10] + "T23:59:59.000Z"
  # get images acquired within a date range
  date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start,
        "lte": end
    }
  }

  # combine our geo, date, cloud filters
  combined_filter = {
    "type": "AndFilter",
    #"config": [geometry_filter]

    "config": [geometry_filter, date_range_filter]
  }

  item_type = "PSScene"

  # API request object
  search_request = {
    "item_types": [item_type], 
    "filter": combined_filter
  }

  # fire off the POST request
  search_result = \
    requests.post(
      'https://api.planet.com/data/v1/quick-search',
      auth=HTTPBasicAuth(API_KEY, ''),
      json=search_request)

  geojson = search_result.json()

  if index % 10 == 0:
    print(index)
  index += 1

  # extract image IDs only
  image_ids = [feature['id'] for feature in geojson['features']]
  if len(image_ids) > 0:
    print(image_ids)
    # For demo purposes, just grab the first image ID
    id0 = image_ids[0]
    id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id0_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    print(result.json().keys())

    print(result.json()['ortho_visual']['status'])
    # Parse out useful links
    links = result.json()[u"ortho_visual"]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the 'ortho_analytic_4b' asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
    )
    activation_status_result = \
    requests.get(
      self_link,
      auth=HTTPBasicAuth(API_KEY, '')
    )
    print(activation_status_result.json)
    print(activation_status_result.json()["status"])
    # Image can be downloaded by making a GET with your Planet API key, from here:
    download_link = activation_status_result.json()["location"]
    print(download_link)

    url = download_link
    myfile = requests.get(url)    
    'c:/users/shash/downloads/PythonImage21.tif'
    lat = round(geo_geom["coordinates"][0], 3)
    long = round(geo_geom["coordinates"][1], 3)
    file_name = "noShip_" + str(lat) + "_" + str(long) + "_" + str(start[:10])
    open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\"+file_name+".tif", 'wb').write(myfile.content)
    """tiff_image = Image.open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".tif")
    png = tiff_image.convert('RGB')
    png = png.convert('L')
    png.show()
    png.save("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".png", "PNG")"""
    image_count += 1
    print(image_count)


In [ ]:
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              38.02810552458331,
              40.95967467099615
            ],
            [
              38.43716585241606,
              40.90815244577769
            ],
            [
              39.15731331880926,
              40.883345116640015
            ],
            [
              39.843794934986875,
              40.89470418769625
            ],
            [
              40.82127349474948,
              40.97827078240374
            ],
            [
              41.46541242975508,
              41.18987076473144
            ],
            [
              41.8250844235736,
              41.441998861270406
            ],
            [
              41.835822402482194,
              41.962201199042
            ],
            [
              41.767241328889924,
              42.42301691593747
            ],
            [
              41.38750518600705,
              42.91440129481208
            ],
            [
              40.89448127051284,
              43.24635798875255
            ],
            [
              40.40129658534781,
              43.36611915849929
            ],
            [
              39.84619036342173,
              43.59704837183881
            ],
            [
              39.601186649251446,
              43.958211272684224
            ],
            [
              39.211138504412105,
              44.15703947088909
            ],
            [
              38.79543277341361,
              44.38052919825458
            ],
            [
              38.150037457591026,
              44.58972186489166
            ],
            [
              37.57312499763526,
              44.76238427143457
            ],
            [
              37.5867118782204,
              44.98714400720357
            ],
            [
              37.18151272670488,
              45.212244156357656
            ],
            [
              36.34633158527055,
              45.34424380802429
            ],
            [
              36.24618904504132,
              45.14249218802644
            ],
            [
              35.87693289235625,
              45.14361420851665
            ],
            [
              35.39047916886361,
              45.19150486207647
            ],
            [
              35.22053536950261,
              45.01988961791352
            ],
            [
              34.72644243947855,
              44.896656264452304
            ],
            [
              34.27316244562789,
              44.664998467745704
            ],
            [
              33.86645457326978,
              44.56201717623844
            ],
            [
              33.67638378118863,
              44.7178223753597
            ],
            [
              33.688064344920946,
              45.066710974365435
            ],
            [
              33.370266286632614,
              45.392741240867934
            ],
            [
              32.573157698833995,
              45.47135013822145
            ],
            [
              32.89262263945392,
              45.677084247149224
            ],
            [
              33.68801966998171,
              45.810285114664765
            ],
            [
              34.484247867558054,
              46.297960445786146
            ],
            [
              33.98509327073816,
              46.341202000478546
            ],
            [
              33.46737878344652,
              46.30565438549783
            ],
            [
              32.95343535378345,
              46.211319082521214
            ],
            [
              32.30625596016117,
              46.19482023778457
            ],
            [
              32.3267486462874,
              46.38423568654824
            ],
            [
              32.549837448691676,
              46.756905355008456
            ],
            [
              32.25807327501795,
              46.9759818817366
            ],
            [
              31.824501768583445,
              46.903383669453774
            ],
            [
              31.098854320222813,
              46.89944285957503
            ],
            [
              30.61986177536494,
              46.66840825236801
            ],
            [
              30.038652676197756,
              46.53320991324702
            ],
            [
              30.00146031043883,
              46.29869802227279
            ],
            [
              29.70754940066584,
              45.93564646196862
            ],
            [
              29.323790968319486,
              45.747090112634936
            ],
            [
              29.277137130739504,
              45.443861140805495
            ],
            [
              29.344103950349762,
              45.121282438708135
            ],
            [
              28.86226566668907,
              45.152069182136756
            ],
            [
              28.564372628949883,
              44.85059949498023
            ],
            [
              28.471086655983584,
              44.45549673540103
            ],
            [
              28.316710841005403,
              44.21146512791162
            ],
            [
              28.420483100120293,
              43.86696984027736
            ],
            [
              28.30789436923689,
              43.54605125850068
            ],
            [
              28.211166992036766,
              43.353870105687804
            ],
            [
              27.738753453221705,
              43.20108269543579
            ],
            [
              27.752049951410243,
              42.967172989087146
            ],
            [
              27.67487043543227,
              42.6892807458847
            ],
            [
              27.430532653329635,
              42.40435990173995
            ],
            [
              27.39278193366576,
              42.16915671981303
            ],
            [
              27.773534848976965,
              41.97738662026711
            ],
            [
              27.907046606891583,
              41.698710486392486
            ],
            [
              28.254080470617083,
              41.35028517698291
            ],
            [
              28.58595983999919,
              41.21614455471362
            ],
            [
              29.75807443731577,
              40.987874505662035
            ],
            [
              30.342616817537902,
              40.97895418493093
            ],
            [
              30.800711594560823,
              40.99523611950181
            ],
            [
              31.155741053398202,
              41.052523672373894
            ],
            [
              31.669268247885128,
              41.28507117736086
            ],
            [
              32.03870968340868,
              41.492576999545264
            ],
            [
              32.307589482356434,
              41.652915600431044
            ],
            [
              32.815436105738,
              41.70134752137923
            ],
            [
              33.14904285065373,
              41.96039672585343
            ],
            [
              33.85066883240512,
              41.90950913561417
            ],
            [
              34.275471752746455,
              41.87353426343492
            ],
            [
              34.4840380287319,
              41.977694567112025
            ],
            [
              35.05719913530649,
              41.850578552532255
            ],
            [
              35.57945598609916,
              41.644173917504105
            ],
            [
              35.988656869086384,
              41.81257522329679
            ],
            [
              36.116542549109965,
              41.57285777465103
            ],
            [
              36.35462348403058,
              41.28124225904659
            ],
            [
              36.518015162252055,
              41.25828684773401
            ],
            [
              36.69942430425266,
              41.373977614463314
            ],
            [
              37.000268790979504,
              41.28318843016979
            ],
            [
              36.95772477263387,
              41.17127761836366
            ],
            [
              38.02810552458331,
              40.95967467099615
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [ ]:
#images using black sea as region
# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

image_count = 1
index = 0
for x in range(1):
  geo_geom = {
    "type": "Polygon",
    "coordinates": [
          [
            [
              38.02810552458331,
              40.95967467099615
            ],
            [
              38.43716585241606,
              40.90815244577769
            ],
            [
              39.15731331880926,
              40.883345116640015
            ],
            [
              39.843794934986875,
              40.89470418769625
            ],
            [
              40.82127349474948,
              40.97827078240374
            ],
            [
              41.46541242975508,
              41.18987076473144
            ],
            [
              41.8250844235736,
              41.441998861270406
            ],
            [
              41.835822402482194,
              41.962201199042
            ],
            [
              41.767241328889924,
              42.42301691593747
            ],
            [
              41.38750518600705,
              42.91440129481208
            ],
            [
              40.89448127051284,
              43.24635798875255
            ],
            [
              40.40129658534781,
              43.36611915849929
            ],
            [
              39.84619036342173,
              43.59704837183881
            ],
            [
              39.601186649251446,
              43.958211272684224
            ],
            [
              39.211138504412105,
              44.15703947088909
            ],
            [
              38.79543277341361,
              44.38052919825458
            ],
            [
              38.150037457591026,
              44.58972186489166
            ],
            [
              37.57312499763526,
              44.76238427143457
            ],
            [
              37.5867118782204,
              44.98714400720357
            ],
            [
              37.18151272670488,
              45.212244156357656
            ],
            [
              36.34633158527055,
              45.34424380802429
            ],
            [
              36.24618904504132,
              45.14249218802644
            ],
            [
              35.87693289235625,
              45.14361420851665
            ],
            [
              35.39047916886361,
              45.19150486207647
            ],
            [
              35.22053536950261,
              45.01988961791352
            ],
            [
              34.72644243947855,
              44.896656264452304
            ],
            [
              34.27316244562789,
              44.664998467745704
            ],
            [
              33.86645457326978,
              44.56201717623844
            ],
            [
              33.67638378118863,
              44.7178223753597
            ],
            [
              33.688064344920946,
              45.066710974365435
            ],
            [
              33.370266286632614,
              45.392741240867934
            ],
            [
              32.573157698833995,
              45.47135013822145
            ],
            [
              32.89262263945392,
              45.677084247149224
            ],
            [
              33.68801966998171,
              45.810285114664765
            ],
            [
              34.484247867558054,
              46.297960445786146
            ],
            [
              33.98509327073816,
              46.341202000478546
            ],
            [
              33.46737878344652,
              46.30565438549783
            ],
            [
              32.95343535378345,
              46.211319082521214
            ],
            [
              32.30625596016117,
              46.19482023778457
            ],
            [
              32.3267486462874,
              46.38423568654824
            ],
            [
              32.549837448691676,
              46.756905355008456
            ],
            [
              32.25807327501795,
              46.9759818817366
            ],
            [
              31.824501768583445,
              46.903383669453774
            ],
            [
              31.098854320222813,
              46.89944285957503
            ],
            [
              30.61986177536494,
              46.66840825236801
            ],
            [
              30.038652676197756,
              46.53320991324702
            ],
            [
              30.00146031043883,
              46.29869802227279
            ],
            [
              29.70754940066584,
              45.93564646196862
            ],
            [
              29.323790968319486,
              45.747090112634936
            ],
            [
              29.277137130739504,
              45.443861140805495
            ],
            [
              29.344103950349762,
              45.121282438708135
            ],
            [
              28.86226566668907,
              45.152069182136756
            ],
            [
              28.564372628949883,
              44.85059949498023
            ],
            [
              28.471086655983584,
              44.45549673540103
            ],
            [
              28.316710841005403,
              44.21146512791162
            ],
            [
              28.420483100120293,
              43.86696984027736
            ],
            [
              28.30789436923689,
              43.54605125850068
            ],
            [
              28.211166992036766,
              43.353870105687804
            ],
            [
              27.738753453221705,
              43.20108269543579
            ],
            [
              27.752049951410243,
              42.967172989087146
            ],
            [
              27.67487043543227,
              42.6892807458847
            ],
            [
              27.430532653329635,
              42.40435990173995
            ],
            [
              27.39278193366576,
              42.16915671981303
            ],
            [
              27.773534848976965,
              41.97738662026711
            ],
            [
              27.907046606891583,
              41.698710486392486
            ],
            [
              28.254080470617083,
              41.35028517698291
            ],
            [
              28.58595983999919,
              41.21614455471362
            ],
            [
              29.75807443731577,
              40.987874505662035
            ],
            [
              30.342616817537902,
              40.97895418493093
            ],
            [
              30.800711594560823,
              40.99523611950181
            ],
            [
              31.155741053398202,
              41.052523672373894
            ],
            [
              31.669268247885128,
              41.28507117736086
            ],
            [
              32.03870968340868,
              41.492576999545264
            ],
            [
              32.307589482356434,
              41.652915600431044
            ],
            [
              32.815436105738,
              41.70134752137923
            ],
            [
              33.14904285065373,
              41.96039672585343
            ],
            [
              33.85066883240512,
              41.90950913561417
            ],
            [
              34.275471752746455,
              41.87353426343492
            ],
            [
              34.4840380287319,
              41.977694567112025
            ],
            [
              35.05719913530649,
              41.850578552532255
            ],
            [
              35.57945598609916,
              41.644173917504105
            ],
            [
              35.988656869086384,
              41.81257522329679
            ],
            [
              36.116542549109965,
              41.57285777465103
            ],
            [
              36.35462348403058,
              41.28124225904659
            ],
            [
              36.518015162252055,
              41.25828684773401
            ],
            [
              36.69942430425266,
              41.373977614463314
            ],
            [
              37.000268790979504,
              41.28318843016979
            ],
            [
              36.95772477263387,
              41.17127761836366
            ],
            [
              38.02810552458331,
              40.95967467099615
            ]
          ]
        ]
  }
  # get images that overlap with our AOI 
  geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_geom
  }
  #2023-06-27T08:33:48.000Z
  time = "2023-06-27"
  start = time + "T00:00:00.000Z"
  end = time + "T23:59:59.000Z"
  # get images acquired within a date range
  date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start,
        "lte": end
    }
  }

  # combine our geo, date, cloud filters
  combined_filter = {
    "type": "AndFilter",
    #"config": [geometry_filter]

    "config": [geometry_filter, date_range_filter]
  }

  item_type = "PSScene"

  # API request object
  search_request = {
    "item_types": [item_type], 
    "filter": combined_filter
  }

  # fire off the POST request
  search_result = \
    requests.post(
      'https://api.planet.com/data/v1/quick-search',
      auth=HTTPBasicAuth(API_KEY, ''),
      json=search_request)

  geojson = search_result.json()

  if index % 10 == 0:
    print(index)
  index += 1

  # extract image IDs only
  image_ids_list = [feature['id'] for feature in geojson['features']]

  shipImages = os.listdir("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\sat_images")
  usedImages = []
  for ship in shipImages:
    usedImages.append(ship[:-4])
  for image_ids in image_ids_list:
    if image_ids not in usedImages:
      # For demo purposes, just grab the first image ID
      id0 = image_ids
      id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
      # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
      result = \
        requests.get(
          id0_url,
          auth=HTTPBasicAuth(API_KEY, '')
        )
      print(result.json().keys())

      print(result.json()['ortho_visual']['status'])
      # Parse out useful links
      links = result.json()[u"ortho_visual"]["_links"]
      self_link = links["_self"]
      activation_link = links["activate"]

      # Request activation of the 'ortho_analytic_4b' asset:
      activate_result = \
        requests.get(
          activation_link,
          auth=HTTPBasicAuth(API_KEY, '')
      )
      activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
      print(activation_status_result.json)
      print(activation_status_result.json()["status"])
      # Image can be downloaded by making a GET with your Planet API key, from here:
      download_link = activation_status_result.json()["location"]
      print(download_link)

      url = download_link
      myfile = requests.get(url)    
      'c:/users/shash/downloads/PythonImage21.tif'
      """    lat = round(geo_geom["coordinates"][0], 3)
      long = round(geo_geom["coordinates"][1], 3)
      file_name = "noShip_" + str(lat) + "_" + str(long) + "_" + str(start[:10])"""
      file_name = str(image_ids)
      open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\sat_images\\"+file_name+".tif", 'wb').write(myfile.content)
      """tiff_image = Image.open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".tif")
      png = tiff_image.convert('RGB')
      png = png.convert('L')
      png.show()
      png.save("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\ship" + str(image_count)+".png", "PNG")"""
      image_count += 1
      print(image_count)


In [ ]:
#gets images from dataset
from PIL import Image
import os
counter = 0
images = os.listdir("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\sat_images")
for image in images:
    im = Image.open("C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\sat_images\\"+image)
    print(im.size)
    png = im.convert('RGB')
    png = png.convert('L')
    box = (5000,5000,5080,5080)
    pngNew = png.crop(box)
    fileName = 'C:\\Users\\shash\\VS Code Programs\\Python\\Research - CNNs\\shipsFromSat\\sat_images\\ships_sat_png\\'  + image[:-4] +".jpg"
    pngNew.save(fileName, 'JPEG', quality=80)
    print(counter)
    counter += 1


